In [ ]:
import numpy as np
import pandas as pd
import napari
from scipy.spatial.transform import Rotation
import tifffile

# Function to apply transformation
def apply_transformation(points, transform):
    rotation = Rotation.from_matrix(transform[:3, :3])
    translation = transform[:3, 3]
    return rotation.apply(points) + translation


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    x, y, z = np.round(point).astype(int)
    if 0 <= x < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= z < mask.shape[2]:
        return mask[x, y, z]
    return False


# Load transformation matrix for lm trials to lm_stack
t_lm_trials_to_lm_stack = np.load("t_lm_trials_to_lm_stack.npy")

# Load centroids of lm plane
lm_trials_centroids = np.load("lm_trials_centroids.npy")

# Load em warped mask
em_warped_mask = tifffile.imread("em_warped_mask.tif")

# Apply transformations to lm_plane_centroids to fit lm_stak and to ultimately fit to em_stack 

centroids_in_lm_stack = apply_transformation(lm_plane_centroids, lm_to_lm_stack)
centroids_in_em_stack = apply_transformation(centroids_in_lm_stack, lm_stack_to_em_stack)


# Identify  centroids within the 3D mask

centroids_in_mask = [point for point in centroids_in_em_stack if is_within_mask(point, em_mask)]


# Validate by visual inspection in napari (Found cells in green)
# Create Napari viewer
viewer = napari.Viewer()

# Add layers to the viewer
viewer.add_image(em_mask, name='EM Mask')
viewer.add_points(points_layer, size=5, face_color='green', name='Found Cells')

# Run Napari viewer
napari.run()
